In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions as distributions

import numpy as np

import visdom
vis = visdom.Visdom()

import time

Setting up a new session...


## Generate and plot some data

In [2]:
normal_distribution = distributions.Normal(0.0, 1)
x_values = torch.arange(1., 30.1).reshape(-1, 1)
class_0_y_values = (1 / 3.) * x_values
class_0_y_values += normal_distribution.sample(class_0_y_values.shape)
class_1_y_values = (1.5) * x_values 
class_1_y_values += normal_distribution.sample(class_1_y_values.shape)
X = torch.cat([torch.cat([x_values, class_0_y_values], axis = 1), torch.cat([x_values, class_1_y_values], axis = 1)], axis = 0) 
y = torch.cat([torch.zeros((int(X.shape[0]/2), 1)), torch.ones((int(X.shape[0]/2), 1))], axis = 0)

np_X = np.array(X)
color = np.zeros((np_X.shape[0], 1))
color[int(color.shape[0] / 2):, :] = 1

In [3]:
vis.scatter(np_X, color + 1, env = "intro", win = "Scatterplot")

'Scatterplot'

In [4]:
vis.line(class_0_y_values.reshape(-1), x_values.reshape(-1), env = "intro", win = "Line", name = "class_0")
vis.line(class_1_y_values.reshape(-1), x_values.reshape(-1), env = "intro", win = "Line", name = "class_1", update = "replace")

'Line'

## Define a neural network with a single neuron

In [5]:
# Definition of the neural network
class SingleNeuronNeuralNetwork(nn.Module):
    def __init__(self):
        super(SingleNeuronNeuralNetwork, self).__init__()
        self.single_neuron_linear_layer = nn.Linear(2, 1, bias = False)

    def forward(self, X):
        return self.single_neuron_linear_layer.forward(X)

In [6]:
neural_network = SingleNeuronNeuralNetwork()
learning_rate = 0.01
optimizer = optim.Adam(neural_network.parameters(), lr = learning_rate)

## Training the neural network

In [7]:
number_of_training_epochs = 1000
for training_epoch in range(number_of_training_epochs):
    # Training loop 
    answers = neural_network.forward(X)
    loss = F.binary_cross_entropy_with_logits(answers, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if training_epoch % 100 == 0:
        # There should be a decrease in the loss! 
        print(f"Training epoch: {training_epoch} - Loss: {loss.item()}")
        
        vis.line(class_0_y_values.reshape(-1), x_values.reshape(-1), env = "intro", win = "Line", name = "class_0", update = "replace")
        vis.line(class_1_y_values.reshape(-1), x_values.reshape(-1), env = "intro", win = "Line", name = "class_1", update = "replace")
        question = (neural_network.single_neuron_linear_layer.weight[0, 1] / neural_network.single_neuron_linear_layer.weight[0, 0] * x_values).reshape(-1)
        vis.line(question, x_values.reshape(-1), env = "intro", win = "Line", name = "question", update = "replace")
        question_perpendicular = -(neural_network.single_neuron_linear_layer.weight[0, 0] / neural_network.single_neuron_linear_layer.weight[0, 1] * x_values).reshape(-1)
        vis.line(question_perpendicular, x_values.reshape(-1), env = "intro", win = "Line", name = "perpendicular_to_question", update = "replace")
        time.sleep(1)

Training epoch: 0 - Loss: 12.54507827758789
Training epoch: 100 - Loss: 0.4178733825683594
Training epoch: 200 - Loss: 0.19788159430027008
Training epoch: 300 - Loss: 0.14432086050510406
Training epoch: 400 - Loss: 0.1178559884428978
Training epoch: 500 - Loss: 0.10133636742830276
Training epoch: 600 - Loss: 0.08971814066171646
Training epoch: 700 - Loss: 0.08093252032995224
Training epoch: 800 - Loss: 0.07395780831575394
Training epoch: 900 - Loss: 0.06822485476732254


In [34]:
# F.sigmoid(neural_network.forward(X))